In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.linear_model import LassoCV

In [2]:
# --- Load your datasets ---
from google.colab import drive
drive.mount('/content/drive')

train_df = pd.read_csv("/content/drive/MyDrive/ADM Final Project/train_v3 (1).csv")
val_df = pd.read_csv("/content/drive/MyDrive/ADM Final Project/test_v3 (1).csv")
test_df = pd.read_csv("/content/drive/MyDrive/ADM Final Project/test__no_lossv3 (1).csv")



Mounted at /content/drive


/tmp/ipython-input-2925491128.py:5: DtypeWarning: Columns (134,203,270) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("/content/drive/MyDrive/ADM Final Project/train_v3 (1).csv")
/tmp/ipython-input-2925491128.py:6: DtypeWarning: Columns (203,270,413) have mixed types. Specify dtype option on import or set low_memory=False.
  val_df = pd.read_csv("/content/drive/MyDrive/ADM Final Project/test_v3 (1).csv")
/tmp/ipython-input-2925491128.py:7: DtypeWarning: Columns (203,270,413) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("/content/drive/MyDrive/ADM Final Project/test__no_lossv3 (1).csv")


In [3]:
train_df.head()

,Unnamed: 0,id,f1,f3,f4,f5,f6,f7,f8,f9,...,f769,f770,f771,f772,f773,f774,f775,f776,f777,loss
0,78539,78539,120,0.146251,2200,4,76878,8703.0,724.0,118.93,...,-5.81,10,3.98,-2.89,2.18,0.2458,0.9434,0,0,0
1,61541,61541,154,0.349399,4200,4,76635,2843.0,3253.0,158.08,...,-5.22,7,4.17,-3.50,3.02,0.1991,0.7517,1,1,0
2,76531,76531,126,0.975969,1500,10,7399,437.0,1453.0,123.31,...,-4.41,12,2.25,-1.35,0.89,0.2290,-0.7491,1,0,0
3,22066,22066,127,0.951303,3100,7,14448,2681.0,2136.0,128.62,...,-6.14,12,4.06,-3.00,2.31,0.2766,0.4056,0,0,11
4,45589,45589,162,0.518486,3500,7,80502,3840.0,130.0,159.78,...,-14.27,28,9.57,-7.15,5.59,0.2864,0.4730,0,1,0


In [4]:
test_df.head()

,Unnamed: 0,id,f1,f3,f4,f5,f6,f7,f8,f9,...,f768,f769,f770,f771,f772,f773,f774,f775,f776,f777
0,7933,7933,119,0.309685,1500,4,2496,89.0,2899.0,121.60,...,-0.4720,-2.36,5,1.51,-1.07,0.80,0.2815,0.5302,1,0
1,101860,101860,117,0.792900,1100,16,842,8117.0,2907.0,124.86,...,-0.5147,-42.72,83,28.13,-20.23,15.27,0.2720,0.6992,0,0
2,62580,62580,129,0.744534,1500,17,14213,5505.0,251.0,127.50,...,-0.2750,-0.55,2,0.17,-0.05,0.02,0.0968,1.5731,1,0
3,1760,1760,159,0.851018,4100,2,79754,2298.0,2188.0,156.44,...,-0.4275,-1.71,4,1.02,-0.74,0.61,0.2688,-0.9013,0,1
4,48008,48008,125,0.495025,1300,15,76957,2816.0,3721.0,128.08,...,-0.4156,-6.65,16,3.80,-2.52,1.78,0.2545,-0.6698,0,0


In [5]:
val_df.head()

,Unnamed: 0,id,f1,f3,f4,f5,f6,f7,f8,f9,...,f769,f770,f771,f772,f773,f774,f775,f776,f777,loss
0,7933,7933,119,0.309685,1500,4,2496,89.0,2899.0,121.60,...,-2.36,5,1.51,-1.07,0.80,0.2815,0.5302,1,0,0
1,101860,101860,117,0.792900,1100,16,842,8117.0,2907.0,124.86,...,-42.72,83,28.13,-20.23,15.27,0.2720,0.6992,0,0,0
2,62580,62580,129,0.744534,1500,17,14213,5505.0,251.0,127.50,...,-0.55,2,0.17,-0.05,0.02,0.0968,1.5731,1,0,0
3,1760,1760,159,0.851018,4100,2,79754,2298.0,2188.0,156.44,...,-1.71,4,1.02,-0.74,0.61,0.2688,-0.9013,0,1,0
4,48008,48008,125,0.495025,1300,15,76957,2816.0,3721.0,128.08,...,-6.65,16,3.80,-2.52,1.78,0.2545,-0.6698,0,0,0


In [6]:
# --- Separate targets and ids ---
train_ids = train_df['id']
val_ids = val_df['id']
test_ids = test_df['id']

y_train = train_df['loss']
y_val = val_df['loss']

# --- Drop id and loss columns ---
X_train = train_df.drop(columns=['id', 'loss'])
X_val = val_df.drop(columns=['id', 'loss'])
X_test = test_df.drop(columns=['id'])

# --- Z-score scaling ---
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_val_scaled = pd.DataFrame(scaler.transform(X_val), columns=X_val.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [7]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LassoCV
from sklearn.decomposition import PCA

# ------------------------
# 1. Remove near-zero variance features
# ------------------------
var_thresh = VarianceThreshold(threshold=0.01)
X_train_nzv = pd.DataFrame(var_thresh.fit_transform(X_train_scaled), columns=X_train_scaled.columns[var_thresh.get_support()])
X_val_nzv = pd.DataFrame(var_thresh.transform(X_val_scaled), columns=X_train_nzv.columns)
X_test_nzv = pd.DataFrame(var_thresh.transform(X_test_scaled), columns=X_train_nzv.columns)

In [8]:
# 2. Impute missing values
# ------------------------
imputer = SimpleImputer(strategy='median')
X_train_imputed = pd.DataFrame(imputer.fit_transform(X_train_nzv), columns=X_train_nzv.columns)
X_val_imputed = pd.DataFrame(imputer.transform(X_val_nzv), columns=X_val_nzv.columns)
X_test_imputed = pd.DataFrame(imputer.transform(X_test_nzv), columns=X_test_nzv.columns)

In [9]:
# 3. Remove highly correlated features
# ------------------------
def remove_high_corr_features(df, threshold=0.98):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [col for col in upper.columns if any(upper[col] > threshold)]
    return df.drop(columns=to_drop), to_drop

X_train_uncorr, dropped_cols = remove_high_corr_features(X_train_imputed, threshold=0.98)
X_val_uncorr = X_val_imputed.drop(columns=dropped_cols)
X_test_uncorr = X_test_imputed.drop(columns=dropped_cols)


In [10]:
# 4. Feature selection using Lasso
# ------------------------
lasso = LassoCV(cv=5, random_state=42,max_iter = 50000).fit(X_train_uncorr, y_train)
selected_features = X_train_uncorr.columns[lasso.coef_ != 0]

X_train_final = X_train_uncorr[selected_features]
X_val_final = X_val_uncorr[selected_features]
X_test_final = X_test_uncorr[selected_features]

In [11]:
X_train_final.shape

(80000, 78)

In [12]:
X_test_final.shape

(25471, 78)

In [13]:
X_val_final.shape

(25471, 78)

In [14]:
# 5. Apply PCA (retain 98% variance)
# ------------------------
pca = PCA(n_components=0.98)
X_train_pca = pd.DataFrame(pca.fit_transform(X_train_final))
X_val_pca = pd.DataFrame(pca.transform(X_val_final))
X_test_pca = pd.DataFrame(pca.transform(X_test_final))

In [15]:
X_train_pca.shape

(80000, 65)

In [16]:
X_test_pca.shape

(25471, 65)

In [17]:
X_val_pca.shape

(25471, 65)

In [18]:
# --- TRAIN ---
train_df = pd.DataFrame(X_train_pca)
train_df['id'] = train_ids.values
train_df['loss'] = y_train.values

# --- VALIDATION ---
val_df = pd.DataFrame(X_val_pca)
val_df['id'] = val_ids.values
val_df['loss'] = y_val.values

# --- TEST (id only) ---
test_df = pd.DataFrame(X_test_pca)
test_df['id'] = test_ids.values


In [19]:
train_df['default'] = (train_df['loss'] > 0).astype(int)
val_df['default'] = (val_df['loss'] > 0).astype(int)

In [20]:
print(train_df.head())

          0         1         2         3         4         5         6  \
0  1.358151  0.614386 -1.944182 -1.268697 -0.839188 -2.810730  1.061213   
1  0.229014 -2.501000  2.432563 -1.948861 -1.621943 -1.645092 -1.241537   
2  3.662303 -0.098241 -1.953701  0.106210  0.616645  0.398768  0.515062   
3 -0.876405 -1.474955 -2.163650  0.761389 -1.561073  4.827997  0.870483   
4 -4.110465  0.320593  2.857586  2.480129 -0.308000 -0.345924 -0.980569   

          7         8         9  ...        58        59        60        61  \
0  1.337513 -0.328538 -0.170003  ...  0.297901 -0.147451  0.807766  1.160649   
1  1.156471 -1.811361  1.377483  ... -0.451020  0.269936  0.656608 -0.581389   
2 -0.537082  1.302052  0.122339  ...  0.681669  0.652199 -0.203750  0.080609   
3 -0.328061 -0.171102 -1.158114  ... -0.593343 -0.126241  0.296488  0.039230   
4  0.672703  0.936000 -1.306312  ...  0.554300  0.113269  0.328277 -0.115816   

         62        63        64     id  loss  default  
0 -0.206039 

In [21]:
print(val_df.head())

          0         1         2         3         4         5         6  \
0  2.057721 -1.218621 -0.115392 -2.465274 -0.681068  0.668913 -1.677200   
1 -4.061519  6.880375 -1.199100 -1.754992 -0.671379 -0.451399  0.145699   
2  3.288247 -0.707657 -2.172725 -0.499418 -0.356455 -0.447060 -0.283055   
3  0.499210 -2.813142  0.583329 -0.889572 -0.435977 -2.004438  0.578724   
4  0.977800  0.687791 -2.061640  0.435974 -0.061519 -0.971846 -1.230245   

          7         8         9  ...        58        59        60        61  \
0 -1.364206  0.047246  0.229200  ...  0.645698  0.040327  0.332478 -0.019688   
1 -1.518834 -1.414918 -0.598659  ... -0.014491 -0.383956  0.356455 -0.323567   
2  0.004001  0.758158  0.793294  ... -0.120092 -0.155444  0.411772 -0.497530   
3  1.029906  0.890776 -1.770457  ...  0.327526  0.146285 -0.360223  0.129716   
4  0.623573 -1.822817  1.129711  ... -0.654166  0.068628  0.196670 -0.961430   

         62        63        64      id  loss  default  
0 -0.175356

In [22]:
print(test_df.head())

          0         1         2         3         4         5         6  \
0  2.057721 -1.218621 -0.115392 -2.465274 -0.681068  0.668913 -1.677200   
1 -4.061519  6.880375 -1.199100 -1.754992 -0.671379 -0.451399  0.145699   
2  3.288247 -0.707657 -2.172725 -0.499418 -0.356455 -0.447060 -0.283055   
3  0.499210 -2.813142  0.583329 -0.889572 -0.435977 -2.004438  0.578724   
4  0.977800  0.687791 -2.061640  0.435974 -0.061519 -0.971846 -1.230245   

          7         8         9  ...        56        57        58        59  \
0 -1.364206  0.047246  0.229200  ... -0.471843 -0.365570  0.645698  0.040327   
1 -1.518834 -1.414918 -0.598659  ...  0.290748 -0.231086 -0.014491 -0.383956   
2  0.004001  0.758158  0.793294  ...  0.335001  0.469347 -0.120092 -0.155444   
3  1.029906  0.890776 -1.770457  ...  0.138695  1.075298  0.327526  0.146285   
4  0.623573 -1.822817  1.129711  ... -0.133682 -0.556260 -0.654166  0.068628   

         60        61        62        63        64      id  
0  0.3

In [23]:
c1def = train_df["default"].values


In [24]:
v1def = val_df["default"].values


# **Logistic Regression**


In [25]:
import numpy as np
import pandas as pd

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, confusion_matrix, classification_report

# -------------------------------------------------------------------------
# 1. Build SMOTE + Logistic Regression pipeline to handle class imbalance
# -------------------------------------------------------------------------
log_reg_model = ImbPipeline(steps=[
    ("smote", SMOTE(random_state=42)),
    ("lr", LogisticRegression(
        class_weight=None,
        max_iter=1000,
        solver="lbfgs"
    ))
])

# -------------------------------------------------------------------------
# 2. Fit model on PCA-transformed training data
# -------------------------------------------------------------------------
log_reg_model.fit(X_train_pca, c1def)

# -------------------------------------------------------------------------
# 3. Predict probabilities on validation set
# -------------------------------------------------------------------------
val_probs = log_reg_model.predict_proba(X_val_pca)[:, 1]   # probability of class 1

# -------------------------------------------------------------------------
# 4. Find best threshold based on F1 score
# -------------------------------------------------------------------------
thresholds = np.linspace(0.05, 0.95, 19)

best_threshold = 0.5
best_f1 = -1

for t in thresholds:
    preds_t = (val_probs >= t).astype(int)
    f1_t = f1_score(v1def, preds_t, pos_label=1)

    if f1_t > best_f1:
        best_f1 = f1_t
        best_threshold = t

print(f"Best F1 threshold: {best_threshold:.2f}")
print(f"Best F1 score: {best_f1:.4f}")

# -------------------------------------------------------------------------
# 5. Final predictions using best threshold
# -------------------------------------------------------------------------
final_preds = (val_probs >= best_threshold).astype(int)

# -------------------------------------------------------------------------
# 6. Confusion Matrix
# -------------------------------------------------------------------------
cm = confusion_matrix(v1def, final_preds)
tn, fp, fn, tp = cm.ravel()

print("\nConfusion Matrix:")
print(cm)
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"True Positives: {tp}")

# -------------------------------------------------------------------------
# 7. Classification Report
# -------------------------------------------------------------------------
print("\nClassification Report:")
print(classification_report(v1def, final_preds, digits=4))

# -------------------------------------------------------------------------
# 8. Extract all True Positive cases
# -------------------------------------------------------------------------
tp_indices = np.where((v1def == 1) & (final_preds == 1))[0]

print(f"\nTotal True Positives found: {len(tp_indices)}")

tp_df = pd.DataFrame({
    "Index_in_Validation": tp_indices,
    "True_Label": v1def[tp_indices],       # fixed: no iloc
    "Predicted_Prob": val_probs[tp_indices]
})


print("\nSample True Positive Cases:")
print(tp_df.head())


Best F1 threshold: 0.55
Best F1 score: 0.2520

Confusion Matrix:
[[16294  6773]
 [ 1081  1323]]
True Negatives: 16294
False Positives: 6773
False Negatives: 1081
True Positives: 1323

Classification Report:
              precision    recall  f1-score   support

           0     0.9378    0.7064    0.8058     23067
           1     0.1634    0.5503    0.2520      2404

    accuracy                         0.6916     25471
   macro avg     0.5506    0.6284    0.5289     25471
weighted avg     0.8647    0.6916    0.7535     25471


Total True Positives found: 1323

Sample True Positive Cases:
   Index_in_Validation  True_Label  Predicted_Prob
0                   10           1        0.562054
1                   37           1        0.758867
2                   43           1        0.643908
3                   68           1        0.583885
4                   72           1        0.648807


In [26]:
import numpy as np
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# -----------------------------------------------------------
# 1. Build confusion matrix
# -----------------------------------------------------------
cm = confusion_matrix(v1def, final_preds)
tn, fp, fn, tp = cm.ravel()

# -----------------------------------------------------------
# 2. Compute all evaluation metrics
# -----------------------------------------------------------
accuracy  = accuracy_score(v1def, final_preds)
precision = precision_score(v1def, final_preds, zero_division=0)
recall    = recall_score(v1def, final_preds)     # TPR / Sensitivity
f1        = f1_score(v1def, final_preds)

TPR = recall                                     # same as recall
FPR = fp / (fp + tn) if (fp + tn) > 0 else 0
FNR = fn / (fn + tp) if (fn + tp) > 0 else 0
TNR = tn / (tn + fp) if (tn + fp) > 0 else 0     # Specificity

# -----------------------------------------------------------
# 3. Create a labeled confusion matrix dataframe
# -----------------------------------------------------------
cm_df = pd.DataFrame(
    cm,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
)

# -----------------------------------------------------------
# 4. Create a side-by-side actual vs predicted table
# -----------------------------------------------------------
results_df = pd.DataFrame({
    "Actual": v1def,
    "Predicted": final_preds,
    "Prob_of_1": val_probs
})

# Optional: results_df.head()

# -----------------------------------------------------------
# 5. Display everything
# -----------------------------------------------------------
print("===== CONFUSION MATRIX =====")
print(cm_df)

print("\n===== METRICS =====")
print(f"Accuracy:      {accuracy:.4f}")
print(f"Precision:     {precision:.4f}")
print(f"Recall (TPR):  {TPR:.4f}")
print(f"FPR:           {FPR:.4f}")
print(f"FNR:           {FNR:.4f}")
print(f"TNR:           {TNR:.4f}")
print(f"F1 Score:      {f1:.4f}")

print("\n===== SAMPLE SIDE-BY-SIDE ACTUAL VS PREDICTED =====")
print(results_df.head())


===== CONFUSION MATRIX =====
          Pred 0  Pred 1
Actual 0   16294    6773
Actual 1    1081    1323

===== METRICS =====
Accuracy:      0.6916
Precision:     0.1634
Recall (TPR):  0.5503
FPR:           0.2936
FNR:           0.4497
TNR:           0.7064
F1 Score:      0.2520

===== SAMPLE SIDE-BY-SIDE ACTUAL VS PREDICTED =====
   Actual  Predicted  Prob_of_1
0       0          0   0.433312
1       0          0   0.453522
2       0          0   0.180462
3       0          0   0.423048
4       0          0   0.292145


# **XGBoost Model**

In [60]:
import numpy as np
from xgboost import XGBClassifier

# Class imbalance handling with scale_pos_weight
pos = (c1def == 1).sum()
neg = (c1def == 0).sum()
scale_pos_weight = neg / pos

xgb_model = XGBClassifier(
    n_estimators=400,
    learning_rate=0.03,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_weight=5,      # prevents overfitting
    gamma=1,
    objective="binary:logistic",
    eval_metric="logloss",
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

from sklearn.calibration import CalibratedClassifierCV

# Fit XGBoost
xgb_model.fit(X_train_final, c1def)

# Calibrate probability output
calibrated_xgb = CalibratedClassifierCV(xgb_model, method='sigmoid')
calibrated_xgb.fit(X_train_final, c1def)

# Use calibrated probabilities
val_probs_xgb = calibrated_xgb.predict_proba(X_val_final)[:, 1]


In [61]:
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# 1. Search for best threshold based on F1 score
thresholds = np.linspace(0.05, 0.95, 19)

best_threshold_xgb = 0.5
best_f1_xgb = -1

for t in thresholds:
    preds_t = (val_probs_xgb >= t).astype(int)
    f1_t = f1_score(v1def, preds_t, pos_label=1)
    if f1_t > best_f1_xgb:
        best_f1_xgb = f1_t
        best_threshold_xgb = t

print(f"Best F1 threshold (XGBoost): {best_threshold_xgb:.2f}")
print(f"Best F1 score (XGBoost): {best_f1_xgb:.4f}")

# 2. Final predictions at best threshold
final_preds_xgb = (val_probs_xgb >= best_threshold_xgb).astype(int)

# 3. Confusion matrix
cm_xgb = confusion_matrix(v1def, final_preds_xgb)
tn, fp, fn, tp = cm_xgb.ravel()

cm_xgb_df = pd.DataFrame(
    cm_xgb,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
)

# 4. Metrics
accuracy_xgb  = accuracy_score(v1def, final_preds_xgb)
precision_xgb = precision_score(v1def, final_preds_xgb, zero_division=0)
recall_xgb    = recall_score(v1def, final_preds_xgb)      # TPR
f1_xgb        = f1_score(v1def, final_preds_xgb)

TPR_xgb = recall_xgb
FPR_xgb = fp / (fp + tn) if (fp + tn) > 0 else 0
FNR_xgb = fn / (fn + tp) if (fn + tp) > 0 else 0
TNR_xgb = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n===== XGBoost Confusion Matrix =====")
print(cm_xgb_df)

print("\n===== XGBoost Metrics =====")
print(f"Accuracy:      {accuracy_xgb:.4f}")
print(f"Precision:     {precision_xgb:.4f}")
print(f"Recall (TPR):  {TPR_xgb:.4f}")
print(f"FPR:           {FPR_xgb:.4f}")
print(f"FNR:           {FNR_xgb:.4f}")
print(f"TNR:           {TNR_xgb:.4f}")
print(f"F1 Score:      {f1_xgb:.4f}")


Best F1 threshold (XGBoost): 0.10
Best F1 score (XGBoost): 0.2605

===== XGBoost Confusion Matrix =====
          Pred 0  Pred 1
Actual 0   15015    8052
Actual 1     838    1566

===== XGBoost Metrics =====
Accuracy:      0.6510
Precision:     0.1628
Recall (TPR):  0.6514
FPR:           0.3491
FNR:           0.3486
TNR:           0.6509
F1 Score:      0.2605


# **LightGBM model**

In [29]:
import numpy as np
from lightgbm import LGBMClassifier

# Handle class imbalance
pos = (c1def == 1).sum()
neg = (c1def == 0).sum()
scale_pos_weight = neg / pos

lgb_model = LGBMClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=31,
    min_child_samples=40,
    reg_alpha=0.1,
    reg_lambda=0.1,

    subsample=0.8,
    colsample_bytree=0.8,
    objective="binary",
    class_weight=None,
    scale_pos_weight=scale_pos_weight,
    random_state=42
)

# Fit on original Lasso-selected features (no PCA)
lgb_model.fit(X_train_final, c1def)

# Predicted probabilities for class 1 on validation set
val_probs_lgb = lgb_model.predict_proba(X_val_final)[:, 1]


[LightGBM] [Info] Number of positive: 7379, number of negative: 72621
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.018189 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 13857
[LightGBM] [Info] Number of data points in the train set: 80000, number of used features: 78
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.092238 -> initscore=-2.286616
[LightGBM] [Info] Start training from score -2.286616


In [30]:
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# 1. Search for best threshold based on F1 score
thresholds = np.linspace(0.05, 0.95, 19)

best_threshold_lgb = 0.5
best_f1_lgb = -1

for t in thresholds:
    preds_t = (val_probs_lgb >= t).astype(int)
    f1_t = f1_score(v1def, preds_t, pos_label=1)
    if f1_t > best_f1_lgb:
        best_f1_lgb = f1_t
        best_threshold_lgb = t

print(f"Best F1 threshold (LightGBM): {best_threshold_lgb:.2f}")
print(f"Best F1 score   (LightGBM): {best_f1_lgb:.4f}")

# 2. Final predictions at best threshold
final_preds_lgb = (val_probs_lgb >= best_threshold_lgb).astype(int)

# 3. Confusion matrix
cm_lgb = confusion_matrix(v1def, final_preds_lgb)
tn, fp, fn, tp = cm_lgb.ravel()

cm_lgb_df = pd.DataFrame(
    cm_lgb,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
)

# 4. Metrics
accuracy_lgb  = accuracy_score(v1def, final_preds_lgb)
precision_lgb = precision_score(v1def, final_preds_lgb, zero_division=0)
recall_lgb    = recall_score(v1def, final_preds_lgb)      # TPR
f1_lgb        = f1_score(v1def, final_preds_lgb)

TPR_lgb = recall_lgb
FPR_lgb = fp / (fp + tn) if (fp + tn) > 0 else 0
FNR_lgb = fn / (fn + tp) if (fn + tp) > 0 else 0
TNR_lgb = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n===== LightGBM Confusion Matrix =====")
print(cm_lgb_df)

print("\n===== LightGBM Metrics =====")
print(f"Accuracy:      {accuracy_lgb:.4f}")
print(f"Precision:     {precision_lgb:.4f}")
print(f"Recall (TPR):  {TPR_lgb:.4f}")
print(f"FPR:           {FPR_lgb:.4f}")
print(f"FNR:           {FNR_lgb:.4f}")
print(f"TNR:           {TNR_lgb:.4f}")
print(f"F1 Score:      {f1_lgb:.4f}")


Best F1 threshold (LightGBM): 0.55
Best F1 score   (LightGBM): 0.2653

===== LightGBM Confusion Matrix =====
          Pred 0  Pred 1
Actual 0   18168    4899
Actual 1    1287    1117

===== LightGBM Metrics =====
Accuracy:      0.7571
Precision:     0.1857
Recall (TPR):  0.4646
FPR:           0.2124
FNR:           0.5354
TNR:           0.7876
F1 Score:      0.2653


# **Bagging with Decision Tree**



In [31]:
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

# Bagging with Decision Trees + SMOTE for imbalance
bag_model = ImbPipeline(steps=[
    ("smote", SMOTE(random_state=42)),
    ("bag", BaggingClassifier(
        estimator=DecisionTreeClassifier(
            max_depth=None,
            class_weight="balanced"   # handle imbalance in base tree
        ),
        n_estimators=200,
        max_samples=1.0,
        max_features=1.0,
        bootstrap=True,
        n_jobs=-1,
        random_state=42
    ))
])

# Fit Bagging on original Lasso-selected features (no PCA)
bag_model.fit(X_train_final, c1def)

# Predict probabilities for validation
val_probs_bag = bag_model.predict_proba(X_val_final)[:, 1]


In [32]:
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# 1. Find best threshold based on F1
thresholds = np.linspace(0.05, 0.95, 19)

best_threshold_bag = 0.5
best_f1_bag = -1

for t in thresholds:
    preds_t = (val_probs_bag >= t).astype(int)
    f1_t = f1_score(v1def, preds_t, pos_label=1)
    if f1_t > best_f1_bag:
        best_f1_bag = f1_t
        best_threshold_bag = t

print(f"Best F1 threshold (Bagging): {best_threshold_bag:.2f}")
print(f"Best F1 score    (Bagging): {best_f1_bag:.4f}")

# 2. Final predictions
final_preds_bag = (val_probs_bag >= best_threshold_bag).astype(int)

# 3. Confusion matrix
cm_bag = confusion_matrix(v1def, final_preds_bag)
tn, fp, fn, tp = cm_bag.ravel()

cm_bag_df = pd.DataFrame(
    cm_bag,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
)

# 4. Metrics
accuracy_bag  = accuracy_score(v1def, final_preds_bag)
precision_bag = precision_score(v1def, final_preds_bag, zero_division=0)
recall_bag    = recall_score(v1def, final_preds_bag)   # TPR
f1_bag        = f1_score(v1def, final_preds_bag)

TPR_bag = recall_bag
FPR_bag = fp / (fp + tn) if (fp + tn) > 0 else 0
FNR_bag = fn / (fn + tp) if (fn + tp) > 0 else 0
TNR_bag = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n===== Bagging Confusion Matrix =====")
print(cm_bag_df)

print("\n===== Bagging Metrics =====")
print(f"Accuracy:      {accuracy_bag:.4f}")
print(f"Precision:     {precision_bag:.4f}")
print(f"Recall (TPR):  {TPR_bag:.4f}")
print(f"FPR:           {FPR_bag:.4f}")
print(f"FNR:           {FNR_bag:.4f}")
print(f"TNR:           {TNR_bag:.4f}")
print(f"F1 Score:      {f1_bag:.4f}")


Best F1 threshold (Bagging): 0.15
Best F1 score    (Bagging): 0.2147

===== Bagging Confusion Matrix =====
          Pred 0  Pred 1
Actual 0   12210   10857
Actual 1     809    1595

===== Bagging Metrics =====
Accuracy:      0.5420
Precision:     0.1281
Recall (TPR):  0.6635
FPR:           0.4707
FNR:           0.3365
TNR:           0.5293
F1 Score:      0.2147


# **Random Forest**

In [33]:
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier

# Random Forest plus SMOTE to handle imbalance
rf_model = ImbPipeline(steps=[
    ("smote", SMOTE(random_state=42)),
    ("rf", RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        min_samples_split=2,
        min_samples_leaf=1,
        max_features="sqrt",
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    ))
])

# Fit on original Lasso-selected features (no PCA)
rf_model.fit(X_train_final, c1def)

# Predicted probabilities for class 1 on validation data
val_probs_rf = rf_model.predict_proba(X_val_final)[:, 1]


In [34]:
import pandas as pd
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# 1. Find best threshold based on F1 score
thresholds = np.linspace(0.05, 0.95, 19)

best_threshold_rf = 0.5
best_f1_rf = -1

for t in thresholds:
    preds_t = (val_probs_rf >= t).astype(int)
    f1_t = f1_score(v1def, preds_t, pos_label=1)
    if f1_t > best_f1_rf:
        best_f1_rf = f1_t
        best_threshold_rf = t

print(f"Best F1 threshold (RF): {best_threshold_rf:.2f}")
print(f"Best F1 score   (RF): {best_f1_rf:.4f}")

# 2. Final predictions at best threshold
final_preds_rf = (val_probs_rf >= best_threshold_rf).astype(int)

# 3. Confusion matrix
cm_rf = confusion_matrix(v1def, final_preds_rf)
tn, fp, fn, tp = cm_rf.ravel()

cm_rf_df = pd.DataFrame(
    cm_rf,
    index=["Actual 0", "Actual 1"],
    columns=["Pred 0", "Pred 1"]
)

# 4. Metrics
accuracy_rf  = accuracy_score(v1def, final_preds_rf)
precision_rf = precision_score(v1def, final_preds_rf, zero_division=0)
recall_rf    = recall_score(v1def, final_preds_rf)   # TPR
f1_rf        = f1_score(v1def, final_preds_rf)

TPR_rf = recall_rf
FPR_rf = fp / (fp + tn) if (fp + tn) > 0 else 0
FNR_rf = fn / (fn + tp) if (fn + tp) > 0 else 0
TNR_rf = tn / (tn + fp) if (tn + fp) > 0 else 0

print("\n===== Random Forest Confusion Matrix =====")
print(cm_rf_df)

print("\n===== Random Forest Metrics =====")
print(f"Accuracy:      {accuracy_rf:.4f}")
print(f"Precision:     {precision_rf:.4f}")
print(f"Recall (TPR):  {TPR_rf:.4f}")
print(f"FPR:           {FPR_rf:.4f}")
print(f"FNR:           {FNR_rf:.4f}")
print(f"TNR:           {TNR_rf:.4f}")
print(f"F1 Score:      {f1_rf:.4f}")


Best F1 threshold (RF): 0.25
Best F1 score   (RF): 0.2271

===== Random Forest Confusion Matrix =====
          Pred 0  Pred 1
Actual 0   15603    7464
Actual 1    1140    1264

===== Random Forest Metrics =====
Accuracy:      0.6622
Precision:     0.1448
Recall (TPR):  0.5258
FPR:           0.3236
FNR:           0.4742
TNR:           0.6764
F1 Score:      0.2271


# **Regression models for predicting "Loss"**

In [35]:
X_train_final.head()

,f3,f5,f36,f53,f55,f65,f67,f70,f80,f82,...,f662,f665,f673,f674,f682,f693,f734,f740,f775,f776
0,-1.222084,-0.651696,-0.398172,-0.840477,-0.133125,0.653059,1.165975,0.984231,1.963319,2.355906,...,-0.538463,-0.530910,-0.358136,1.598526,-0.448284,-0.449675,-0.038778,0.103371,0.893575,-0.669947
1,-0.517746,-0.651696,-0.560925,-0.822645,0.130236,0.666469,-1.837054,-1.221328,0.194916,2.298994,...,2.057776,-0.631424,0.492907,-0.808170,2.806894,-0.305166,-0.624665,0.379249,0.708886,1.492655
2,1.654647,0.513581,-0.289671,-0.600568,-1.029787,-0.346107,1.293832,0.814572,-0.689285,-0.546612,...,-1.295279,-0.515940,-0.323230,-0.655364,-0.031165,-0.622155,0.184805,-0.414904,-0.737025,1.492655
3,1.569126,-0.069057,-0.289671,-0.935842,0.664959,0.677641,0.022827,-0.882011,-0.367757,-0.546612,...,-0.511277,-0.392614,-0.308270,-0.196946,-0.645841,-0.263988,0.374117,0.179413,0.375444,-0.669947
4,0.068497,-0.069057,0.578340,1.459921,-0.182596,-0.329971,-1.072831,0.305597,0.596826,0.136334,...,1.649551,0.793595,0.034142,0.910899,-0.142197,1.035821,0.840868,0.394042,0.440379,-0.669947


In [36]:
# Train set: selected features + id + loss
train_data = X_train_final.copy()
train_data["id"] = train_ids.values
train_data["loss"] = y_train.values

# Keep only defaulters (loss > 0)
train_data = train_data[train_data["loss"] > 0].reset_index(drop=True)

# Validation set: selected features + id + loss
val_data = X_val_final.copy()
val_data["id"] = val_ids.values
val_data["loss"] = y_val.values

# Keep only defaulters (loss > 0)
val_data = val_data[val_data["loss"] > 0].reset_index(drop=True)


# Test set: selected features + id only
test_data = X_test_final.copy()
test_data["id"] = test_ids.values


In [37]:
train_data.head()

,f3,f5,f36,f53,f55,f65,f67,f70,f80,f82,...,f673,f674,f682,f693,f734,f740,f775,f776,id,loss
0,1.569126,-0.069057,-0.289671,-0.935842,0.664959,0.677641,0.022827,-0.882011,-0.367757,-0.546612,...,-0.308270,-0.196946,-0.645841,-0.263988,0.374117,0.179413,0.375444,-0.669947,22066,11
1,0.300013,1.873071,0.198585,-0.714371,0.846110,0.784940,1.116563,-0.033720,-0.287376,-0.546612,...,-0.298297,1.713131,-0.360759,0.260441,0.346373,0.880136,0.745881,-0.669947,32243,5
2,-1.508226,-0.651696,-0.560925,-0.868972,1.180404,0.840252,0.011259,0.814572,0.034152,-0.205139,...,0.060737,-0.235147,-0.158202,-0.411606,0.659716,-0.280729,0.934809,-0.669947,93195,4
3,0.699619,-0.651696,-0.669426,-0.671827,0.145150,0.104464,-0.613508,-1.560645,-0.247185,0.876191,...,0.010871,-0.464356,-0.023163,-0.681978,-1.603863,-0.380647,-0.466013,-0.669947,94740,7
4,-0.434488,1.484645,-0.723677,-0.772738,-0.911929,-0.584415,-0.073466,0.644914,-0.407948,-0.375875,...,0.993228,-0.769968,1.423499,-0.804734,-0.314586,-0.461619,-1.255638,-0.669947,42822,14


In [38]:
val_data.head()

,f3,f5,f36,f53,f55,f65,f67,f70,f80,f82,...,f673,f674,f682,f693,f734,f740,f775,f776,id,loss
0,0.351681,-0.845908,-0.018417,1.315623,0.009105,-0.942691,-0.925779,0.475255,0.034152,1.217664,...,-0.343176,0.032264,1.033388,-0.159879,-0.851513,-1.152481,-0.632204,1.492655,9308,4
1,0.736529,-0.069057,0.469839,1.724139,0.115322,-1.167693,-2.055917,-1.221328,0.355680,-0.546612,...,-0.334866,1.178309,-0.465039,0.371543,-1.352536,-1.261742,0.160792,-0.669947,63146,3
2,1.458293,-0.651696,-0.832178,0.701704,-1.493941,0.676919,-0.719574,0.475255,0.998736,2.355906,...,-0.291648,-0.999178,-0.308244,-0.713056,0.219077,-0.190673,1.670771,1.492655,60383,1
3,0.696990,0.513581,-0.723677,-0.995570,1.053452,1.009994,-0.049048,-0.542695,-0.206994,-0.546612,...,-0.253418,-0.311550,-0.763374,-0.633032,-1.228504,-0.485755,1.096761,-0.669947,99848,7
4,0.646230,-0.845908,0.795343,-0.718320,0.264099,0.357506,2.083403,0.135939,-0.247185,-0.375875,...,0.010871,1.980541,-0.719362,0.173425,0.791908,2.305201,-0.444240,-0.669947,18661,23


In [39]:
# Regression Modeling on Lasso Features

from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import lightgbm as lgb

# -------------------------------------------
# 1. Prepare X and y for modeling
# -------------------------------------------
X_train_reg = train_data.drop(columns=['id', 'loss'])
X_val_reg = val_data.drop(columns=['id', 'loss'])
X_test_reg = test_data.drop(columns=['id'])

y_train_reg = train_data['loss']
y_val_reg = val_data['loss']

import numpy as np

# Log-transform the target to reduce skew
y_train_reg_log = np.log1p(y_train_reg)
# Noise smoothing to stabilize tree regressors
y_train_reg_log += np.random.normal(0, 1e-4, size=len(y_train_reg_log))
y_val_reg_log   = np.log1p(y_val_reg)

print("Train shape:", X_train_reg.shape)
print("Validation shape:", X_val_reg.shape)

Train shape: (7379, 78)
Validation shape: (2404, 78)


**Elastic Net Model**

In [40]:
# MODEL 1: Elastic Net (automatic lambda + alpha selection)

elastic_model = ElasticNetCV(
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    cv=5,
    random_state=42,
    max_iter=20000
)

# Fit on log-transformed target
elastic_model.fit(X_train_reg, y_train_reg_log)

# --- Predict on validation (log scale) ---
elastic_pred_val_log = elastic_model.predict(X_val_reg)

# Transform predictions back to original loss scale
elastic_pred_val = np.expm1(elastic_pred_val_log)

# Clip negative values
elastic_pred_val = np.clip(elastic_pred_val, 0, None)

# --- Metrics on original scale ---
rmse_elastic = mean_squared_error(y_val_reg, elastic_pred_val) ** 0.5
mae_elastic  = mean_absolute_error(y_val_reg, elastic_pred_val)
r2_elastic   = r2_score(y_val_reg, elastic_pred_val)

print("========== ELASTIC NET RESULTS ==========")
print("Best Alpha (λ):", elastic_model.alpha_)
print("Best L1 Ratio:", elastic_model.l1_ratio_)
print("RMSE:", rmse_elastic)
print("MAE:", mae_elastic)
print("R²:", r2_elastic)

========== ELASTIC NET RESULTS ==========
Best Alpha (λ): 0.0010427657402329279
Best L1 Ratio: 0.9
RMSE: 9.597289943949319
MAE: 4.8206311441412835
R²: 0.26318499523424044


**XGB Regressor Model**

In [41]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# ====== MODEL: XGBoost Regressor (original loss, no scaling) ======
xgb_reg = XGBRegressor(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=4,
    subsample=0.8,
    colsample_bytree=0.8,
    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1
)

# ====== TRAIN ======
# Train on log(loss + 1)
xgb_reg.fit(X_train_reg, y_train_reg_log)

# Predict on validation (log scale)
xgb_pred_val_log = xgb_reg.predict(X_val_reg)

# Back-transform to original scale
xgb_pred_val = np.expm1(xgb_pred_val_log)

# Clip negatives
xgb_pred_val = np.clip(xgb_pred_val, 0, None)

# Metrics on original scale
rmse_xgb = mean_squared_error(y_val_reg, xgb_pred_val) ** 0.5
mae_xgb  = mean_absolute_error(y_val_reg, xgb_pred_val)
r2_xgb   = r2_score(y_val_reg, xgb_pred_val)

print("========== XGBOOST REGRESSOR==========")
print("RMSE:", rmse_xgb)
print("MAE: ", mae_xgb)
print("R²:  ", r2_xgb)


========== XGBOOST REGRESSOR (ORIGINAL LOSS, NO SCALING) ==========
RMSE: 9.367171617610596
MAE:  4.682839870452881
R²:   0.2980952262878418


**SVM Regressor Model**

In [42]:
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# ====== MODEL: SVR WITH RBF KERNEL ======
svr_model = SVR(
    kernel="rbf",
    C=10.0,
    epsilon=0.1,
    gamma="scale"
)

# ====== TRAIN (on log-transformed loss) ======
svr_model.fit(X_train_reg, y_train_reg_log)

# ====== Predict on validation (log scale) ======
svr_pred_val_log = svr_model.predict(X_val_reg)

# ====== Convert back to original loss scale ======
svr_pred_val = np.expm1(svr_pred_val_log)

# Clip negatives
svr_pred_val = np.clip(svr_pred_val, 0, None)

# ====== Metrics ======
rmse_svr = mean_squared_error(y_val_reg, svr_pred_val) ** 0.5
mae_svr  = mean_absolute_error(y_val_reg, svr_pred_val)
r2_svr   = r2_score(y_val_reg, svr_pred_val)

print("========== SVR (RBF) RESULTS ==========")
print("RMSE:", rmse_svr)
print("MAE :", mae_svr)
print("R²  :", r2_svr)


========== SVR (RBF) RESULTS ==========
RMSE: 9.648431069111364
MAE : 5.127894690345984
R²  : 0.2553115339707269


**LightGBM Regressor Model**

In [43]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# ====== MODEL: LightGBM Regressor (original target) ======
lgb_reg = LGBMRegressor(
    n_estimators=800,
    learning_rate=0.02,
    num_leaves=31,
    max_depth=-1,
    subsample=0.8,
    colsample_bytree=0.8,
    min_child_samples=40,    # regularization
    reg_alpha=0.1,           # L1 penalty
    reg_lambda=0.1,          # L2 penalty
    objective="regression",
    random_state=42
)

#Train
lgb_reg.fit(X_train_reg, y_train_reg_log)

lgb_pred_val_log = lgb_reg.predict(X_val_reg)

# Back-transform
lgb_pred_val = np.expm1(lgb_pred_val_log)

# Clip negatives
lgb_pred_val = np.clip(lgb_pred_val, 0, None)

rmse_lgb = mean_squared_error(y_val_reg, lgb_pred_val) ** 0.5
mae_lgb  = mean_absolute_error(y_val_reg, lgb_pred_val)
r2_lgb   = r2_score(y_val_reg, lgb_pred_val)


print("========== LIGHTGBM REGRESSOR (ORIGINAL TARGET) ==========")
print("RMSE:", rmse_lgb)
print("MAE :", mae_lgb)
print("R²  :", r2_lgb)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001965 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 12729
[LightGBM] [Info] Number of data points in the train set: 7379, number of used features: 78
[LightGBM] [Info] Start training from score 1.868811
========== LIGHTGBM REGRESSOR (ORIGINAL TARGET) ==========
RMSE: 9.319429218716186
MAE : 4.690466414734834
R²  : 0.3052319174748157


**Catboost Regressor Model**

In [44]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 24.9 MB/s eta 0:00:00


In [45]:
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

cat_reg = CatBoostRegressor(
    iterations=1200,
    learning_rate=0.03,
    depth=6,
    loss_function="RMSE",
    random_seed=42,
    verbose=0
)

# TRAIN (on log-transformed loss)
cat_reg.fit(X_train_reg, y_train_reg_log)

# Predict on validation (log scale)
cat_pred_val_log = cat_reg.predict(X_val_reg)

# Convert to original scale
cat_pred_val = np.expm1(cat_pred_val_log)

# Clip negatives
cat_pred_val = np.clip(cat_pred_val, 0, None)

# Metrics
rmse_cat = mean_squared_error(y_val_reg, cat_pred_val) ** 0.5
mae_cat  = mean_absolute_error(y_val_reg, cat_pred_val)
r2_cat   = r2_score(y_val_reg, cat_pred_val)

print("========== CATBOOST REGRESSOR ==========")
print("RMSE:", rmse_cat)
print("MAE :", mae_cat)
print("R²  :", r2_cat)


========== CATBOOST REGRESSOR ==========
RMSE: 9.270443105517543
MAE : 4.616790185480632
R²  : 0.31251659982814717


Random Forest Regressor Model

In [46]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# MODEL: Random Forest Regressor on Lasso-selected features
rf_reg = RandomForestRegressor(
    n_estimators=800,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=3,
    max_features="sqrt",
    n_jobs=-1,
    random_state=42
)

# TRAIN (on log-transformed loss)
rf_reg.fit(X_train_reg, y_train_reg_log)

# Predict on validation (log scale)
rf_pred_val_log = rf_reg.predict(X_val_reg)

# Convert back to original scale
rf_pred_val = np.expm1(rf_pred_val_log)

# Clip negatives
rf_pred_val = np.clip(rf_pred_val, 0, None)

# Metrics
rmse_rf = mean_squared_error(y_val_reg, rf_pred_val) ** 0.5
mae_rf  = mean_absolute_error(y_val_reg, rf_pred_val)
r2_rf   = r2_score(y_val_reg, rf_pred_val)

print("========== RANDOM FOREST REGRESSOR (LASSO FEATURES) ==========")
print("RMSE:", rmse_rf)
print("MAE :", mae_rf)
print("R²  :", r2_rf)


========== RANDOM FOREST REGRESSOR (LASSO FEATURES) ==========
RMSE: 10.011486451110208
MAE : 4.913459745454981
R²  : 0.19821420383850108


Comparision of Regression Models

In [47]:
import pandas as pd

results_table = pd.DataFrame([
    ["Elastic Net", rmse_elastic, mae_elastic, r2_elastic],
    ["XGBoost Regressor", rmse_xgb, mae_xgb, r2_xgb],
    ["SVR (RBF)", rmse_svr, mae_svr, r2_svr],
    ["LightGBM Regressor", rmse_lgb, mae_lgb, r2_lgb],
    ["CatBoost Regressor", rmse_cat, mae_cat, r2_cat],
    ["Random Forest Regressor", rmse_rf, mae_rf, r2_rf],
], columns=["Model", "RMSE", "MAE", "R²"])

print(results_table.sort_values("RMSE"))


                     Model       RMSE       MAE        R²
4       CatBoost Regressor   9.270443  4.616790  0.312517
3       LightGBM Regressor   9.319429  4.690466  0.305232
1        XGBoost Regressor   9.367172  4.682840  0.298095
0              Elastic Net   9.597290  4.820631  0.263185
2                SVR (RBF)   9.648431  5.127895  0.255312
5  Random Forest Regressor  10.011486  4.913460  0.198214


# **Using LightGBM Model to predict defaulters on test set**

In [62]:
# Predict class labels (0 or 1) on the PCA-transformed test set
test_preds_lgb_pca = lgb_model.predict(X_test_final)

# View prediction distribution
print("Predicted classes (LightGBM on PCA features):")
print(pd.Series(test_preds_lgb_pca).value_counts())


Predicted classes (LightGBM on PCA features):
0    17261
1     8210
Name: count, dtype: int64


In [63]:
# Combine test features with IDs
X_test_final_with_ids = X_test_final.copy()
X_test_final_with_ids['CustomerID'] = test_ids.values  # Ensure test_ids aligns with X_test_final
X_test_final_with_ids['PredictedDefault'] = test_preds_lgb_pca  # LightGBM predictions on PCA data

# Extract rows where prediction == 1
predicted_defaulters = X_test_final_with_ids[X_test_final_with_ids['PredictedDefault'] == 1]

# View or export
print(predicted_defaulters.head())
predicted_defaulters.to_csv("predicted_defaulters_lgb_final.csv", index=False)


         f3        f5       f36       f53       f55       f65       f67  \
0 -0.655438 -0.651696 -0.669426 -0.464862 -0.856638 -0.033643  0.501505   
5  0.351681 -0.845908 -0.018417  1.315623  0.009105 -0.942691 -0.925779   
7 -0.159802 -0.845908  1.120847 -0.657040 -0.040003  0.455304  0.339970   
8  0.956074  1.678858  0.035833 -0.703814  1.683844  0.476490 -0.889569   
9  1.364087 -1.234334 -0.343922  1.380089 -0.786069 -0.037204 -0.639303   

        f70       f80       f82  ...      f673      f674      f682      f693  \
0 -0.542695 -1.171577 -0.546612  ... -0.235134 -0.235147  0.256917 -0.707617   
5  0.475255  0.034152  1.217664  ... -0.343176  0.032264  1.033388 -0.159879   
7  1.662864 -0.206994  0.876191  ... -0.391380  0.528883 -0.735366  0.554899   
8  0.475255 -0.247185 -0.546612  ... -0.344839  0.185070  0.396206  0.351343   
9  0.135939 -0.126612 -0.148227  ... -0.143713 -0.349752  0.232910  0.072424   

       f734      f740      f775      f776  CustomerID  PredictedDefa

**Using CatBoost Regressor Model to predict loss for predicted defaulters on test set**

In [65]:
import pandas as pd
import numpy as np

# === Step 1: Drop non-feature columns ===
X_defaulters_for_loss = predicted_defaulters.drop(['CustomerID', 'PredictedDefault'], axis=1)

# === Step 2: Predict log(loss + 1) ===
pred_loss_log = cat_reg.predict(X_defaulters_for_loss)

# === Step 3: Convert to original scale ===
pred_loss = np.expm1(pred_loss_log)
pred_loss = np.clip(pred_loss, 0, None)

# === Step 4: Prepare final DataFrame ===
loss_output_df = predicted_defaulters[['CustomerID']].copy()
loss_output_df['PredictedLoss'] = pred_loss

# === Step 5: Save to CSV ===
csv_path = "predicted_loss_defaulters_catboost.csv"
loss_output_df.to_csv(csv_path, index=False)

# === Step 6: Display preview ===
print("First 6 rows of the saved CSV:")
print(loss_output_df.head(6))


First 6 rows of the saved CSV:
    CustomerID  PredictedLoss
0         7933      16.149737
5         9308       4.035240
7        87760       9.764262
8        53334       0.989841
9        72303       8.633624
11       36604       3.549605


In [66]:
from google.colab import files
files.download("predicted_loss_defaulters_catboost.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>